# Dataset Generation for Model Training

First, we add all the useful dependencies for image collection and processing.  
Be sure to be in the root directory of the project.

In [1]:
import cv2
import time
import os
import uuid
from IPython.display import clear_output

In [2]:
imagePath = 'Tensorflow/Workspace/images/capturedimages'

Now we define the list of gesture classes _labels_, which will be used to generate data for the model to train onto.  
The _nImages_ variable specifies how many images of a single gesture shall be taken: the higher, the better variance you'll get in the final dataset. However, be warned that all the images for the dataset must be labeled manually, so choose wisely. We will exploit data augmentation to generate a bigger dataset with more variance, hopefully improving the final model's generalization capabilities.  

In [4]:
labels = ['LeftL', 'LeftA', 'LeftO', 'LeftV', 'LeftOpenHand', 'RIndexUp', 'RIndexDown', 'RIndexLeft', 'RIndexRight', 'RIndexFront', 'RThumbBack', 'RClearC']
nImages = 20